In [2]:
## imports ##
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder 
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [4]:
cd ../


C:\Users\otiwa


In [9]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 90F5-4C94

 Directory of C:\Users\otiwa

12-12-2022  20:50    <DIR>          .
10-10-2022  17:15    <DIR>          ..
11-10-2022  22:45    <DIR>          .anaconda
26-10-2022  15:16    <DIR>          .conda
11-10-2022  22:44                25 .condarc
11-10-2022  22:44    <DIR>          .continuum
22-11-2022  18:08    <DIR>          .ipynb_checkpoints
11-10-2022  22:50    <DIR>          .ipython
11-10-2022  22:48    <DIR>          .jupyter
27-10-2022  21:22    <DIR>          .matplotlib
03-11-2022  17:02            60,604 1_a_i.csv
03-11-2022  17:00            60,589 1_a_ii.csv
03-11-2022  17:13            60,573 1_b_i.csv
03-11-2022  17:09            60,579 1_b_ii.csv
03-11-2022  17:52                67 1_d.csv
04-11-2022  14:57           164,864 12. quetion.ipynb
04-11-2022  18:48           217,471 13. question.ipynb
04-11-2022  18:57            63,983 14.question.ipynb
05-11-2022  16:54            71,657 15. and 16. Question dat

In [23]:
df = pd.read_csv("avila-tr.txt", header = None)
data = df[~(df == '?').any(axis=1)]
data

,0,1,2,3,4,5,6,7,8,9,10
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,A
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,A
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,A
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,A
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,F
...,...,...,...,...,...,...,...,...,...,...,...
10425,0.080916,0.588093,0.015130,0.002250,0.261718,-0.557133,0.371178,0.932346,0.282354,-0.580141,F
10426,0.253730,-0.338346,0.352988,-1.154243,0.172340,-0.557133,0.257927,0.348428,0.032902,-0.527134,F
10427,0.229043,-0.000745,0.171611,-0.002793,0.261718,0.688613,0.295677,-1.088486,-0.590727,0.580142,A
10428,-0.301743,0.352558,0.288973,1.638181,0.261718,0.688613,0.069175,0.502761,0.625350,0.718969,E


In [39]:
## function to generate bootstrap datasets ##
def make_bootstraps(data,n_bootstraps=10):
    #initialize output dictionary & unique value count
    dc   = {}
    unip = 0
    #get sample size
    b_size = data.shape[0]
    #get list of row indexes
    idx = [i for i in range(b_size)]
    #loop through the required number of bootstraps
    for b in range(n_bootstraps):
        #obtain boostrap samples with replacement
        sidx   = np.random.choice(idx,replace=True,size=b_size)
        b_samp = data.iloc[sidx,:]
        #compute number of unique values contained in the bootstrap sample
        unip  += len(set(sidx))
        #obtain out-of-bag samples for the current b
        oidx   = list(set(idx) - set(sidx))
        o_samp = np.array([])
        if oidx:
            o_samp = data.iloc[oidx,:]
        #store results
        dc['boot_'+str(b)] = {'boot':b_samp,'test':o_samp}
        print('training set: ', b_samp.shape)
        print('test set:', o_samp.shape)
    #state the mean number of unique values in the bootstraps
    print('Mean number of unique values in each bootstrap: {:.2f}'.format(unip/n_bootstraps))
    #return the bootstrap results
    return(1)

make_bootstraps(data)

training set:  (10430, 11)
test set: (3824, 11)
training set:  (10430, 11)
test set: (3812, 11)
training set:  (10430, 11)
test set: (3804, 11)
training set:  (10430, 11)
test set: (3787, 11)
training set:  (10430, 11)
test set: (3878, 11)
training set:  (10430, 11)
test set: (3825, 11)
training set:  (10430, 11)
test set: (3834, 11)
training set:  (10430, 11)
test set: (3801, 11)
training set:  (10430, 11)
test set: (3783, 11)
training set:  (10430, 11)
test set: (3760, 11)
Mean number of unique values in each bootstrap: 6619.20


1

In [5]:
df = pd.read_csv("bank-full.csv", sep=';')
#names=['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
data = df[~(df == '?').any(axis=1)]

print(data)
data = data.apply(LabelEncoder().fit_transform)

#discretatizing the age 
data['age'].max()
# data['age'].min()
def change(x):
    if x >= data['age'].min() and x < 50:
        return 1
    elif x >= 50 and x < 70:
        return 2
    elif x >= 70 and x < 80:
        return 3
    elif x >= 80 and x < data['age'].max():
        return 4

data['age'] = data['age'].apply(change)


# normalizing the dataset
def normalize(X):
    mean_X=np.mean(X,axis=0)
    std_X=np.std(X,axis=0)
    X=(X-mean_X)/std_X
    return X

data_x = normalize(data.iloc[:,:-1])

data_y = data.iloc[:, -1]

#the concatinated output
data_copy = pd.concat([data_x, data_y], axis=1)

data_copy

       age           job   marital  education default  balance housing loan  \
0       58    management   married   tertiary      no     2143     yes   no   
1       44    technician    single  secondary      no       29     yes   no   
2       33  entrepreneur   married  secondary      no        2     yes  yes   
3       47   blue-collar   married    unknown      no     1506     yes   no   
4       33       unknown    single    unknown      no        1      no   no   
...    ...           ...       ...        ...     ...      ...     ...  ...   
45206   51    technician   married   tertiary      no      825      no   no   
45207   71       retired  divorced    primary      no     1729      no   no   
45208   72       retired   married  secondary      no     5715      no   no   
45209   57   blue-collar   married  secondary      no      668      no   no   
45210   37  entrepreneur   married  secondary      no     2971      no   no   

         contact  day month  duration  campaign  pd

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,-0.118028,-0.103820,-0.275762,1.036362,-0.13549,0.732955,0.893915,-0.436803,1.514306,-1.298476,0.823773,0.023623,-0.572970,-0.414317,-0.305354,0.444898,0
1,-0.118028,1.424008,1.368372,-0.300556,-0.13549,-0.695795,0.893915,-0.436803,1.514306,-1.298476,0.823773,-0.435364,-0.572970,-0.414317,-0.305354,0.444898,0
2,-0.118028,-0.714951,-0.275762,-0.300556,-0.13549,-0.714243,0.893915,2.289359,1.514306,-1.298476,0.823773,-0.748309,-0.572970,-0.414317,-0.305354,0.444898,0
3,-0.118028,-1.020516,-0.275762,2.373280,-0.13549,0.312051,0.893915,-0.436803,1.514306,-1.298476,0.823773,-0.681548,-0.572970,-0.414317,-0.305354,0.444898,0
4,-0.118028,2.035139,1.368372,2.373280,-0.13549,-0.714927,-1.118674,-0.436803,1.514306,-1.298476,0.823773,-0.239251,-0.572970,-0.414317,-0.305354,0.444898,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,-0.118028,1.424008,-0.275762,1.036362,-0.13549,-0.151899,-1.118674,-0.436803,-0.713012,0.143418,1.156344,3.002866,0.077253,-0.414317,-0.305354,0.444898,1
45207,8.116768,0.201746,-1.919895,-1.637474,-0.13549,0.461691,-1.118674,-0.436803,-0.713012,0.143418,1.156344,0.837282,-0.247859,-0.414317,-0.305354,0.444898,1
45208,8.116768,0.201746,-0.275762,-0.300556,-0.13549,2.385823,-1.118674,-0.436803,-0.713012,0.143418,1.156344,3.591204,0.727475,1.453270,1.292364,-0.566175,1
45209,-0.118028,-1.020516,-0.275762,-0.300556,-0.13549,-0.259175,-1.118674,-0.436803,0.400647,0.143418,1.156344,1.054258,0.402364,-0.414317,-0.305354,0.444898,0


In [6]:
## function to generate bootstrap datasets ##
def make_bootstraps(data,n_bootstraps=10):
    #initialize output dictionary & unique value count
    dc   = {}
    unip = 0
    #get sample size
    b_size = data.shape[0]
    #get list of row indexes
    idx = [i for i in range(b_size)]
    #loop through the required number of bootstraps
    for b in range(n_bootstraps):
        #obtain boostrap samples with replacement
        sidx   = np.random.choice(idx,replace=True,size=b_size)
        b_samp = data.iloc[sidx,:]
        #compute number of unique values contained in the bootstrap sample
        unip  += len(set(sidx))
        #obtain out-of-bag samples for the current b
        oidx   = list(set(idx) - set(sidx))
        o_samp = np.array([])
        if oidx:
            o_samp = data.iloc[oidx,:]
        #store results
        dc['boot_'+str(b)] = {'boot':b_samp,'test':o_samp}
#         print('training set: ', b_samp.shape)
#         print('test set:', o_samp.shape)
    #state the mean number of unique values in the bootstraps
    print('Mean number of unique values in each bootstrap: {:.2f}'.format(unip/n_bootstraps))
    #return the bootstrap results
    return(dc, n_bootstraps)

dc, bootstraps = make_bootstraps(data_copy)

Mean number of unique values in each bootstrap: 28565.30


In [54]:
#for each bootstrap model train and store the result in the dictionay
class naivebayes:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)
#         self.posteriors_arg = []
#         print(n_classes)
        # define the mean, varience and priors
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors = np.zeros(n_classes, dtype=np.float64)
        
        for i in self._classes:
            X_c = X[i==y]
            self._mean[i, :] = X_c.mean(axis=0)
            self._var[i, :] = X_c.var(axis=0)
            self.priors[i] = X_c.shape[0] / float(n_samples)
    
    def prediction(self, X):
#         for i in range(0, X.shape[0]):
#             print(X.iloc[i, :])
#         for x in X:
#             print(x)
        y_pred = [self._prediction(x) for x in X]
        return y_pred
    
    def _prediction(self, x):
        posteriors = []
        
        for idx, c in enumerate(self._classes):
            prior = np.log(self.priors[idx])
            class_conditional = np.log(self._pdf(idx, x))
#             print(class_conditional)
            posterior = prior + class_conditional
            posteriors.append(posterior)
#         print(posteriors)
#         self.posteriors_arg.append(np.argmax(posteriors))
#         print(len(self.posteriors_arg))
        return self._classes[np.argmax(posteriors)]
            
    def _pdf(self, class_id, x):
        mean = self._mean[class_id]
        var = self._var[class_id]
        numerator = np.exp(-(x - mean)**2 / (2* var))
        demoninator = np.sqrt(2 * np.pi * var)
#         print(numerator / demoninator)
        return numerator / demoninator

In [57]:
class NaiveBayes:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y == c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


In [59]:
#developing model for bootstrap data models

#for each model the predicted class
model_op = []

for i in range(bootstraps):
    X = dc['boot_'+str(i)]['boot'].iloc[:, :-1].to_numpy()
    y = dc['boot_'+str(i)]['boot'].iloc[:, -1].to_numpy()
    X_test = dc['boot_'+str(i)]['test'].iloc[:, :-1].to_numpy()
#     print(len(X_test))
    y_test = dc['boot_'+str(i)]['test'].iloc[:, -1].to_numpy()
#     print(X[-1])
    #model training 
    nb = NaiveBayes()
    nb.fit(X, y)
#     prediction1 = nb._prediction(X_test[456])
    predict = nb.predict(X_test)
#     prediction1 = nb._prediction(X[-1])
#     print(prediction1)
    model_op.append(predict)
    
print(model_op)

[array([0, 0, 1, ..., 0, 1, 0]), array([1, 1, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), array([0, 1, 0, ..., 1, 0, 0]), array([1, 0, 0, ..., 0, 1, 0]), array([0, 0, 0, ..., 1, 0, 0]), array([0, 1, 0, ..., 1, 0, 0]), array([0, 0, 0, ..., 1, 0, 0]), array([1, 0, 0, ..., 0, 1, 0]), array([0, 0, 0, ..., 0, 0, 0])]


In [ ]:
#error calculation 
def que1(y_pred, y):
    # for calculating the value of sensitivity, specificity, precision, fmeasure
    true_positive = 0
    false_negative = 0
    true_negative = 0
    false_positive = 0

    for i, j in zip(y, y_pred):
        if i == 1 and i == int(j):
            true_positive += 1
        elif i == 1 and i != int(j):
            false_negative += 1
        elif i == 0 and i == int(j):
            true_negative += 1
        elif i == 0 and i != int(j):
            false_positive += 1
        else:
            continue 

    sensitivity = (true_positive + 1) / ((true_positive + false_negative) + 2)
    specificity = (true_negative + 1)/ ((true_negative + false_positive) + 2)
    precision = (true_positive + 1) / ((true_positive + false_positive) + 2)
    fmeasure = 2*(sensitivity * precision) / (sensitivity + precision)

    return sensitivity, specificity, precision, fmeasure